# Postgres Integration with DuckDB

We gave you already a short introduction in the previous section and how it integrates with pandas. In this section we will show you how to integrate DuckDB with Postgres. You can quite easily use DuckDB as your tool of choice to query a postgres database inside a juptyer notebook and combine it with pandas if you want to.

We loaded the pokemon data via Airbyte into a postgres database. We will now connect to this database and query it with DuckDB.

In [1]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

### Configure the notebook
Set configrations on the ipython sql extension to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

We now have to create a in memory database first and than install the postgres and the postgres_scanner extension for duckdb:

In [3]:
%%sql

duckdb:///:memory:

In [5]:
%%sql
INSTALL postgres;
INSTALL postgres_scanner;
LOAD postgres;
LOAD postgres_scanner;

,Success


And now we can connect to the postgres database and query it:

With the `ATTACH` command we can attach the postgres database to the duckdb database. We can now query the postgres database with the `pokemon` alias.

In [6]:
%%sql

ATTACH 'host=localhost port=5432 user=postgres password=postgres dbname=pokemon' AS pokemon (TYPE postgres);
USE pokemon;

,Success


In [9]:
%%sql

SELECT * FROM pokemon__masterdata LIMIT 10;

,#,hp,Name,speed,total,attack,Type 1,Type 2,defense,Sp. Atk,Sp. Def,legendary,generation,_airbyte_ab_id,_airbyte_emitted_at,_airbyte_normalized_at,_airbyte_pokemon_hashid
0,1.0,45.0,Bulbasaur,45.0,318.0,49.0,Grass,Poison,49.0,65.0,65.0,False,1.0,4f41ed9b-28b0-43d8-92a6-7ef692ea9116,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,465628febd57e4c95a32eb1583bfa97a
1,2.0,60.0,Ivysaur,60.0,405.0,62.0,Grass,Poison,63.0,80.0,80.0,False,1.0,41048d3c-0149-462e-9a87-275c4a6aab8d,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,ef70df0dee0d05f496dcd3b8ccf99e87
2,3.0,80.0,Venusaur,80.0,525.0,82.0,Grass,Poison,83.0,100.0,100.0,False,1.0,d78731dc-3a95-4a49-809c-31ccc88fa49f,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,aac885f0c49fbd788dd76192a15001bd
3,3.0,80.0,VenusaurMega Venusaur,80.0,625.0,100.0,Grass,Poison,123.0,122.0,120.0,False,1.0,cd96d0b1-1955-455a-99fa-9813694dd36c,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,18dda442f04aeea9c82910c0fdae8159
4,4.0,39.0,Charmander,65.0,309.0,52.0,Fire,None,43.0,60.0,50.0,False,1.0,2651a6f3-34d2-42d3-bc6a-65e48d21f375,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,7ccdcf6c9b3b5f2c2995438d3d7a5481
5,5.0,58.0,Charmeleon,80.0,405.0,64.0,Fire,None,58.0,80.0,65.0,False,1.0,45e42e5c-6d8c-4d38-8c74-92fbf5eef324,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,bea0a54f50fb2f9d450908878a4523aa
6,6.0,78.0,Charizard,100.0,534.0,84.0,Fire,Flying,78.0,109.0,85.0,False,1.0,5ab9d6d5-33c2-47a3-b1b3-f20a4b04640f,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,45bc994c5d59cf02560d47b53062b8b5
7,6.0,78.0,CharizardMega Charizard X,100.0,634.0,130.0,Fire,Dragon,111.0,130.0,85.0,False,1.0,99e6d5ef-f336-4eee-81de-84acdbacf81d,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,4d5aa17ed95e820b823d11fed5a49e67
8,6.0,78.0,CharizardMega Charizard Y,100.0,634.0,104.0,Fire,Flying,78.0,159.0,115.0,False,1.0,444fcba3-a6a5-4c6e-abc4-53563e78b96f,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,e42d1c208596d8866a8e6d28a13196df
9,7.0,44.0,Squirtle,43.0,314.0,48.0,Water,None,65.0,50.0,64.0,False,1.0,748b0d92-b821-4189-bfed-6a3de5566928,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,16aae1e1a31f853ae0e6b63ca3e2b8ac


And for example put it into a pandas dataframe:

In [10]:
%%sql

pokemon_masterdata << SELECT * FROM pokemon__masterdata;

In [13]:
pokemon_masterdata.head()

,#,hp,Name,speed,total,attack,Type 1,Type 2,defense,Sp. Atk,Sp. Def,legendary,generation,_airbyte_ab_id,_airbyte_emitted_at,_airbyte_normalized_at,_airbyte_pokemon_hashid
0,1.0,45.0,Bulbasaur,45.0,318.0,49.0,Grass,Poison,49.0,65.0,65.0,False,1.0,4f41ed9b-28b0-43d8-92a6-7ef692ea9116,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,465628febd57e4c95a32eb1583bfa97a
1,2.0,60.0,Ivysaur,60.0,405.0,62.0,Grass,Poison,63.0,80.0,80.0,False,1.0,41048d3c-0149-462e-9a87-275c4a6aab8d,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,ef70df0dee0d05f496dcd3b8ccf99e87
2,3.0,80.0,Venusaur,80.0,525.0,82.0,Grass,Poison,83.0,100.0,100.0,False,1.0,d78731dc-3a95-4a49-809c-31ccc88fa49f,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,aac885f0c49fbd788dd76192a15001bd
3,3.0,80.0,VenusaurMega Venusaur,80.0,625.0,100.0,Grass,Poison,123.0,122.0,120.0,False,1.0,cd96d0b1-1955-455a-99fa-9813694dd36c,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,18dda442f04aeea9c82910c0fdae8159
4,4.0,39.0,Charmander,65.0,309.0,52.0,Fire,None,43.0,60.0,50.0,False,1.0,2651a6f3-34d2-42d3-bc6a-65e48d21f375,2024-01-10 12:47:20+01:00,2024-01-10 12:48:52.873313+01:00,7ccdcf6c9b3b5f2c2995438d3d7a5481
